# Dogs vs. Cats Redux: Kernels Edition - implemented from scratch

### First get standard boilerplate imports out of the way

In [2]:
%matplotlib inline

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

### Define path and batch_size for desired data source

In [126]:
#path = "data/"
path = "data/sample/"
test_path = "data/sample/test"
batch_size = 24

## Create VGG16 model and load pre-trained weights from ImageNet

In [121]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.optimizers import SGD, RMSprop, Adam

### Preprocess routine to center each image pixel around dataset RGB mean value, and reverse axis from RGB -> BGR

In [72]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

In [73]:
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]

In [104]:
model = Sequential()


### Add Input Layer for images with dimension 244 x 244

In [105]:
model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))

### Add Convolutional Layers

In [106]:
def ConvBlock(mymodel, layers, filters):
    for i in range(layers):
        mymodel.add(ZeroPadding2D((1,1)))
        mymodel.add(Convolution2D(filters, 3, 3, activation='relu'))
    mymodel.add(MaxPooling2D((2,2), strides=(2,2)))

In [107]:
ConvBlock(model, 2, 64)
ConvBlock(model, 2, 128)
ConvBlock(model, 3, 256)
ConvBlock(model, 3, 512)
ConvBlock(model, 3, 512)

### Add Fully Connected Layers

In [108]:
def FCBlock(mymodel):
    mymodel.add(Dense(4096, activation='relu'))
    mymodel.add(Dropout(0.5))

In [109]:
model.add(Flatten())
FCBlock(model)
FCBlock(model)

### Add final output Dense Layer

In [110]:
model.add(Dense(1000, activation='softmax'))

### Download and load pre-trained model weights from Imagenet

In [111]:
fname = 'vgg16.h5'
baseurl = 'http://www.platform.ai/models/'
local_weights = get_file(fname, baseurl+fname, cache_subdir='models')

In [112]:
model.load_weights(local_weights)

## Finetune VGG16 by replacing last layer with a Dense layer

In [113]:
model.pop()

Make all remaining layers read-only, since we want to leverage all the pre-trained weights

In [114]:
for layer in model.layers: layer.trainable=False

Get batch of data for training and validation

In [115]:
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
    return gen.flow_from_directory(path, target_size=(224,224), class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [116]:
train_batches = get_batches(path+'train', batch_size=batch_size)
valid_batches = get_batches(path+'valid', batch_size=batch_size)

Found 200 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


### Now add new Dense layer with number of classes we are training with, and compile.

In [117]:
model.add(Dense(train_batches.nb_class, activation='softmax'))

In [124]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Now fit the model with training and validation batches

In [125]:
model.fit_generator(train_batches, 
                    samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=1,
                    validation_data=valid_batches,
                    nb_val_samples=valid_batches.nb_sample
                   )

Epoch 1/1
200/200 [==============================] - 83s - loss: 1.2287 - acc: 0.6750 - val_loss: 0.2153 - val_acc: 0.9500
